In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100.0%


In [4]:
image, label = train_data[0]

In [5]:
image.size()

torch.Size([3, 32, 32])

In [6]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [7]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        #capa convolucional
        self.conv1 = nn.Conv2d(3,12,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(64):
    print(f"Training epoch: {epoch}...")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()
        
        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
 
    print (f"Loss: {running_loss/len(train_loader):.4f}")

Training epoch: 0...
Loss: 2.2402
Training epoch: 1...
Loss: 1.8105
Training epoch: 2...
Loss: 1.5537
Training epoch: 3...
Loss: 1.4084
Training epoch: 4...
Loss: 1.3113
Training epoch: 5...
Loss: 1.2306
Training epoch: 6...
Loss: 1.1569
Training epoch: 7...
Loss: 1.0927
Training epoch: 8...
Loss: 1.0389
Training epoch: 9...
Loss: 0.9910
Training epoch: 10...
Loss: 0.9475
Training epoch: 11...
Loss: 0.9080
Training epoch: 12...
Loss: 0.8697
Training epoch: 13...
Loss: 0.8409
Training epoch: 14...
Loss: 0.8060
Training epoch: 15...
Loss: 0.7722
Training epoch: 16...
Loss: 0.7441
Training epoch: 17...
Loss: 0.7171
Training epoch: 18...
Loss: 0.6905
Training epoch: 19...
Loss: 0.6642
Training epoch: 20...
Loss: 0.6383
Training epoch: 21...
Loss: 0.6156
Training epoch: 22...
Loss: 0.5910
Training epoch: 23...
Loss: 0.5664
Training epoch: 24...
Loss: 0.5472
Training epoch: 25...
Loss: 0.5283
Training epoch: 26...
Loss: 0.5064
Training epoch: 27...
Loss: 0.4820
Training epoch: 28...
Loss: 0.

In [10]:
torch.save(net.state_dict(), 'trained_net.pth')

In [11]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [12]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Accuracy: {accuracy}%")

Accuracy: 65.72%


In [14]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)

    return image

image_paths = ['a350.jpg', 'golden.jpg', 'kimi.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"Prediction: {class_names[predicted.item()]}")

Prediction: car
Prediction: dog
Prediction: dog
